# Genetic Algorithm approach for solving the problem of Project 2.

## Genetic Algorithm proposal:

- **Genetic Algorithm** is a search heuristic that is inspired by Charles Darwin’s theory of natural evolution. This algorithm reflects the process of natural selection where the fittest individuals are selected for reproduction in order to produce offspring of the next generation.
- The algorithm starts with a population of solutions (represented by chromosomes) and applies the principle of natural selection to evolve the population of solutions.
    - For our case, chromosomes will represent the variables used in the model.
    - Given the size of the problem, we will first manually select the best 50 variables using the best performing feature selection method.
    - The genetic algorithm will then be used to select the best subset of these 50 variables.
    - We will also choose the model "manually" during the selection of the best 50 variables.
- The genetic algorithm will be implemented as follows:
    - The fitness function will be the profit function.
    - The genetic operators will be mutation and crossover.
    - The selection operator will be the tournament selection.
    - The stopping criterion will be the number of generations or the number of generations without improvement.
    - The initial population will be generated randomly.
    - The mutation probability will be set to 0.1, and the crossover probability will be set to 0.9. 


In [65]:
import pickle

with open('selected_features_per_method.pkl', 'rb') as f:
    selected_features = pickle.load(f)

selected_features

{'mrmr': Index([105,  64, 131, 136, 100, 102,  24, 103, 104, 359, 101,  29, 266,  57,
         39, 241, 351, 155, 335, 442],
       dtype='int64'),
 'surf': Index([105, 102, 101, 321, 100, 289, 283, 374, 254, 311,
        ...
        378, 228, 354, 394, 232, 332, 299, 279, 356, 382],
       dtype='int64', length=490),
 'forest': Index([100, 101, 102, 103, 104, 105, 403], dtype='int64'),
 'xgb': Index([100, 101, 102, 103, 104, 105], dtype='int64'),
 'common_features': [105.0,
  64.0,
  131.0,
  136.0,
  100.0,
  102.0,
  24.0,
  103.0,
  104.0,
  359.0,
  101.0,
  29.0,
  266.0,
  57.0,
  39.0,
  241.0,
  351.0,
  155.0,
  335.0,
  442.0,
  403.0]}

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import cleanup_dataset_apply_standard_scaler

X = pd.read_table('./data/x_train.txt', header=None, delimiter=' ')
y = pd.read_table('./data/y_train.txt', header=None)

X_test = pd.read_table('./data/x_test.txt', header=None, delimiter=' ')

X.head()

correlated_features_to_remove = [1, 0, 4, 6, 7, 3, 5, 8, 9, 2]
X = X.drop(correlated_features_to_remove, axis=1)
X_test = X_test.drop(correlated_features_to_remove, axis=1)

X = cleanup_dataset_apply_standard_scaler(X)
X_test = cleanup_dataset_apply_standard_scaler(X_test)

In [46]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
from sklearn.ensemble import RandomForestClassifier
from itertools import combinations

common_features = selected_features['common_features']
common_features_sixes_permutations = (
    list(combinations(common_features, 3))
    + list(combinations(common_features, 2))
    + list(combinations(common_features, 1))
)

len(common_features_sixes_permutations)

1561

In [21]:
from tqdm import tqdm
feature_selection_scores = []
current_max_score = 0
current_best_feature_selection = None

tqdm._instances.clear()
bar = tqdm(total=len(common_features_sixes_permutations))
for feature_selection in common_features_sixes_permutations:
    X_train_selected = X_train[pd.Index(feature_selection)]
    X_val_selected = X_val[pd.Index(feature_selection)]

    clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
    clf.fit(X_train_selected, y_train.values.ravel())
    score = clf.score(X_val_selected, y_val)
    feature_selection_scores.append((feature_selection, score))
    
    bar.update(1)
    current_max_score = max(current_max_score, score)
    current_best_feature_selection = feature_selection if score == current_max_score else current_best_feature_selection
    bar.set_description(f'Score: {score}, Max score: {current_max_score}, Best feature selection: {current_best_feature_selection}')


Score: 0.502, Max score: 0.662, Best feature selection: (105.0, 100.0, 102.0, 101.0): 100%|██████████| 7546/7546 [17:19<00:00,  7.26it/s]
Score: 0.502, Max score: 0.634, Best feature selection: (105.0, 100.0, 102.0):  83%|████████▎ | 1302/1561 [02:38<00:31,  8.20it/s]

KeyboardInterrupt: 

In [66]:
from utils import compute_score

best_features = [100, 102, 105, 101, 103, 136, 104, 64, 131]
best_features_combinations = (
    list(combinations(best_features, i))
    for i in range(2, len(best_features) + 1)
)

best_features_combinations = [
    feature_selection 
    for feature_selections in best_features_combinations 
    for feature_selection in feature_selections
]

feature_selection_scores = []
current_max_score = 0
current_best_feature_selection = None

tqdm._instances.clear()
bar = tqdm(total=len(best_features_combinations))
for feature_selection in best_features_combinations:
    X_train_selected = X_train[pd.Index(feature_selection)]
    X_val_selected = X_val[pd.Index(feature_selection)]
    clf = RandomForestClassifier(
        n_estimators=500, 
        max_depth=9, 
        max_features=1,
        bootstrap=True,
        random_state=0, 
        n_jobs=-1
    )
    clf.fit(X_train_selected, y_train.values.ravel())
    
    predicted = clf.predict(X_val_selected)
    
    # Intersection of predicted and actual values where both are 1
    correct_instances_num = len(np.intersect1d(np.where(predicted == 1), np.where(y_val == 1)))
    score = compute_score(correct_instances_num, len(feature_selection))
    
    feature_selection_scores.append((feature_selection, score))
    current_max_score = max(current_max_score, score)
    current_best_feature_selection = feature_selection if score == current_max_score else current_best_feature_selection
    
    bar.update(1)
    bar.set_description(f'Score: {score}, Max score: {current_max_score}, Best feature selection: {current_best_feature_selection}')
    
bar.close()

ImportError: cannot import name 'compute_score' from 'utils' (/Users/nk2/workspace/AdvMachineLearning/aml_projects/Project2/data/utils.py)

In [54]:
best_best_features = [100, 102, 105, 101, 103, 104, 64]

# Grid search best hyperparameters
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': np.arange(100, 1000, 100),
    'max_depth': np.arange(1, 30, 1),
    'max_features': np.arange(1, 15, 1),
    'bootstrap': [True, False]
}

clf = RandomForestClassifier(random_state=0, n_jobs=1)
grid_search = GridSearchCV(clf, param_grid, cv=5, n_jobs=-1, verbose=4)
grid_search.fit(X_train[pd.Index(best_best_features)], y_train.values.ravel())
grid_search.best_params_

Fitting 5 folds for each of 7308 candidates, totalling 36540 fits
[CV 1/5] END bootstrap=True, max_depth=1, max_features=1, n_estimators=200;, score=0.632 total time=   0.2s
[CV 1/5] END bootstrap=True, max_depth=1, max_features=1, n_estimators=500;, score=0.650 total time=   0.7s
[CV 4/5] END bootstrap=True, max_depth=1, max_features=1, n_estimators=700;, score=0.613 total time=   0.8s
[CV 1/5] END bootstrap=True, max_depth=1, max_features=2, n_estimators=100;, score=0.604 total time=   0.1s
[CV 3/5] END bootstrap=True, max_depth=1, max_features=2, n_estimators=100;, score=0.615 total time=   0.2s
[CV 1/5] END bootstrap=True, max_depth=1, max_features=2, n_estimators=200;, score=0.605 total time=   0.2s
[CV 3/5] END bootstrap=True, max_depth=1, max_features=2, n_estimators=300;, score=0.621 total time=   0.4s
[CV 4/5] END bootstrap=True, max_depth=1, max_features=2, n_estimators=500;, score=0.613 total time=   0.6s
[CV 1/5] END bootstrap=True, max_depth=1, max_features=2, n_estimators

KeyboardInterrupt: 